<a href="https://colab.research.google.com/github/Mojtaba-Choopani/huggingface-llm-course-fa-notebooks/blob/main/chapter2-USING-TRANSFORMERS/section4-Handling-multiple-sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">

<b style="font-size: 18px;"> مدیریت چند دنباله </b>

</div>
# Handling multiple sequences (PyTorch)

<div dir="rtl">

<p>
<b>مدل‌ها انتظار دسته‌ای از ورودی‌ها را دارند</b>
</p>

<p>
در بخش قبلی دیدیم که چگونه دنباله‌های متنی به لیستی از اعداد (توکن‌ها) تبدیل می‌شوند. حالا می‌خواهیم این لیست را به <b>تنسور</b> تبدیل کرده و به مدل بدهیم، چون <b>مدل‌ها انتظار دارند ورودی‌ها به‌صورت دسته‌ای (batch)</b> باشند.
</p>

</div>


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

IndexError: too many indices for tensor of dimension 1

<div dir="rtl">

<p>
<b>خطای بالا</b> به این دلیل است که ما فقط <b>یک دنباله (جمله)</b> را به مدل فرستادیم، در حالی که مدل‌های Transformers به‌طور پیش‌فرض انتظار دریافت <b>چندین دنباله (batch)</b> دارند.
</p>

<p>
در واقع، در اینجا ما سعی کردیم همه کارهایی را که توکنایزر به‌صورت پنهانی انجام می‌دهد، خودمان به‌صورت دستی انجام دهیم؛ اما <b>فراموش کردیم</b> که ورودی مدل باید <b>شکل دسته‌ای (batched)</b> داشته باشد، حتی اگر فقط شامل یک جمله باشد.
</p>

</div>


In [4]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


<div dir="rtl">

<p>
<b>ایجاد دسته‌ (batching)</b> به مدل این امکان را می‌دهد که چندین جمله را به‌طور هم‌زمان پردازش کند.
</p>

<p>
اما یک <b>مشکل</b>:
برای پردازش هم‌زمان چند جمله (<code>batching</code>)، همه دنباله‌ها باید <b>هم‌اندازه</b> باشند. چون مدل‌ها فقط <b>تنسورهای مستطیلی</b> را قبول می‌کنند، دنباله‌های کوتاه‌تر را با <b>توکن <code>padding</code></b> پر می‌کنیم تا قابل تبدیل به تنسور شوند.
</p>

</div>


In [6]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

In [7]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

<div dir="rtl">

<p>
<b>شناسهٔ توکن پد (padding)</b> را می‌توان از طریق <code>tokenizer.pad_token_id</code> به‌دست آورد.
</p>

<p>
حالا بیایید از آن استفاده کنیم و <b>دو جمله‌مان را هم به‌صورت جداگانه</b> و هم <b>در قالب یک batch</b> به مدل بفرستیم.
</p>

</div>


In [8]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


<div dir="rtl">

<p>
مشکلی در <b>خروجی <code>logits</code> برای پیش‌بینی‌های دسته‌ای (batched)</b> وجود دارد:
ردیف دوم <code>logits</code> باید با خروجی جمله دوم (در حالت جداگانه) یکسان باشد، اما <b>کاملاً متفاوت</b> است!
</p>

<p>
<b>دلیل چیست؟</b><br>
مدل‌های ترنسفورمر دارای <b>لایه‌های attention</b> هستند که هر توکن را در <b>بافت (context)</b> سایر توکن‌ها تفسیر می‌کنند.
وقتی جمله‌ها به‌صورت batch و با <code>padding</code> ارسال می‌شوند، مدل به همهٔ توکن‌ها — از جمله <b>توکن‌های padding</b> — توجه می‌کند.
</p>

<p>
<b>راه‌حل چیست؟</b><br>
باید به مدل بگوییم که <b>توکن‌های padding را نادیده بگیرد</b>.
</p>

</div>


<div dir="rtl">

<p>
<code>attention_mask</code> توکن‌های مهم را با <b>۱</b> و توکن‌های <code>padding</code> را با <b>۰</b> مشخص می‌کند تا مدل فقط به <b>بخش‌های واقعی جمله</b> توجه کند و <b>padding</b> را نادیده بگیرد.
</p>

</div>


In [9]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [ ]:
sequence = sequence[:max_sequence_length]